In [11]:
import pandas as pd
import json

# Load the tokenized data
data = pd.read_csv('../data/tokenized_data_v3.csv', sep='|')

# Parse the JSON strings to lists
data['X'] = data['X'].apply(json.loads)
data['y'] = data['y'].apply(json.loads)

# Find rows where the lengths of X and y are not equal
unequal_length_rows = data[data.apply(lambda row: len(row['X']) >256, axis=1)]

# Display the rows with unequal lengths
print(unequal_length_rows)


                                                       X  \
19932  [83, 2246, 115, 2412, 2412, 3855, 76, 378, 744...   
19933  [10, 2412, 3855, 76, 378, 744, 266, 1446, 44, ...   
19934  [10, 3855, 76, 378, 744, 266, 1446, 44, 3370, ...   
19935  [5588, 76, 378, 744, 266, 1446, 44, 3370, 9610...   
19936  [68, 744, 266, 1446, 44, 3370, 9610, 662, 1317...   
...                                                  ...   
30502  [261, 294, 109, 4378, 7318, 403, 111, 264, 482...   
30503  [2105, 111, 264, 4828, 5278, 7260, 111, 1221, ...   
30504  [111, 4828, 5278, 7260, 111, 1221, 397, 354, 9...   
30505  [109, 412, 5278, 7260, 111, 1221, 397, 354, 97...   
30506  [288, 103, 5789, 111, 1221, 397, 354, 97, 265,...   

                                                       y  
19932  [10, 2412, 3855, 76, 378, 744, 266, 1446, 44, ...  
19933  [10, 3855, 76, 378, 744, 266, 1446, 44, 3370, ...  
19934  [5588, 76, 378, 744, 266, 1446, 44, 3370, 9610...  
19935  [68, 744, 266, 1446, 44, 3370, 9610,

In [12]:
data[data.apply(lambda row: len(row['X']) >512, axis=1)]

,X,y


In [2]:
# Debug: Print the first few rows to verify the parsing``
print("First few rows after parsing:")
print(data.head())

# Find rows where the lengths of X and y are not equal
def check_unequal_lengths(row):
    len_x = len(row['X'])
    len_y = len(row['y'])
    if len_x != len_y:
        print(f"Unequal lengths found: X={len_x}, y={len_y}, row_index={row.name}")
    return len_x != len_y

unequal_length_rows = data[data.apply(check_unequal_lengths, axis=1)]

# Display the rows with unequal lengths
print("Rows with unequal lengths:")
print(unequal_length_rows)


First few rows after parsing:
                                                   X  \
0  [5758, 410, 2412, 551, 962, 1056, 2412, 2412, ...   
1  [3519, 2412, 551, 962, 1056, 2412, 2412, 706, ...   
2  [10, 551, 962, 1056, 2412, 2412, 706, 8114, 41...   
3  [69, 962, 1056, 2412, 2412, 706, 8114, 410, 33...   
4  [108, 1280, 2412, 2412, 706, 8114, 410, 331, 7...   

                                                   y  
0  [3519, 2412, 551, 962, 1056, 2412, 2412, 706, ...  
1  [10, 551, 962, 1056, 2412, 2412, 706, 8114, 41...  
2  [69, 962, 1056, 2412, 2412, 706, 8114, 410, 33...  
3  [108, 1280, 2412, 2412, 706, 8114, 410, 331, 7...  
4  [10, 2412, 706, 8114, 410, 331, 7013, 325, 32,...  
Rows with unequal lengths:
Empty DataFrame
Columns: [X, y]
Index: []


In [6]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, max_length):
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]
        input_ids = text['X']
        target_ids = text['y']
        
        input_ids_tensor = torch.tensor(input_ids, dtype=torch.long)
        target_ids_tensor = torch.tensor(target_ids, dtype=torch.long)

        return input_ids_tensor, target_ids_tensor


In [7]:
from torch.nn.utils.rnn import pad_sequence
special_tokens = {'<eos>': 10000, '<pad>': 10001}
def collate_fn(batch):
    # Separate the input and target sequences
    input_ids, target_ids = zip(*batch)
    
    # Pad the sequences
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=special_tokens['<pad>'])
    target_ids_padded = pad_sequence(target_ids, batch_first=True, padding_value=special_tokens['<pad>'])
    
    return input_ids_padded, target_ids_padded


In [10]:
import pandas as pd
import json
from torch.utils.data import DataLoader

# Initialize tokenizer and other necessary parameters
# tokenizer = ...  # Initialize your tokenizer
special_tokens = {'<pad>': 10001, '<eos>': 10000}
max_length = 50

# Create dataset and dataloader
dataset = CustomDataset(data, max_length)
dataloader = DataLoader(dataset, batch_size=32, collate_fn=collate_fn)

# Debugging within DataLoader loop
for batch in dataloader:
    input_ids, target_ids = batch
    if input_ids.shape != target_ids.shape:
        print("Input IDs shape:", input_ids.shape)
        print("Target IDs shape:", target_ids.shape)
        print("First batch of Input IDs:", input_ids[0])
        print("First batch of Target IDs:", target_ids[0])
        # break  # Only checking the first batch for debugging
